<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/QMC_Multimodal_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install --upgrade  git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

In [2]:
import tensorflow as tf
import numpy as np
import qmc.tf.layers as layers
import qmc.tf.models as models

In [44]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000,784))
X_test = X_test.reshape((10000,784))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

transformer = OneHotEncoder(sparse=False)
y_train_bin = transformer.fit_transform(y_train[:, np.newaxis])
y_test_bin = transformer.fit_transform(y_test[:, np.newaxis])

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

shape X_train :  (60000, 784)
shape y_train :  (60000,)
shape X_test :  (10000, 784)
shape y_test :  (10000,)


## Baseline

In [42]:
BATCH_SIZE = 256

model = tf.keras.models.Sequential([
tf.keras.layers.Dense(128,activation='relu'),
tf.keras.layers.Dense(10),
tf.keras.layers.Softmax()
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)
EPOCHS = 10
  
history = model.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 1s 3ms/step - loss: 0.8470 - categorical_accuracy: 0.7691 - val_loss: 0.2471 - val_categorical_accuracy: 0.9310
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2399 - categorical_accuracy: 0.9350 - val_loss: 0.1864 - val_categorical_accuracy: 0.9478
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1763 - categorical_accuracy: 0.9504 - val_loss: 0.1547 - val_categorical_accuracy: 0.9568
Epoch 4/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1419 - categorical_accuracy: 0.9597 - val_loss: 0.1356 - val_categorical_accuracy: 0.9609
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1147 - categorical_accuracy: 0.9677 - val_loss: 0.1273 - val_categorical_accuracy: 0.9643
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0947 - categorical_accuracy: 0.9734 - val_loss: 0.1119 - val_categorical_accuracy: 0.9670
Epoc

## Half model

In [5]:
BATCH_SIZE = 256
def create_model(input_dim,
               output_dim,
               num_rff = 512,
               gamma = 2**-5,
               n_comp = 80,
               random_state = 0):
    inputs = tf.keras.Input(shape=(input_dim,))
    fm_x1 = layers.QFeatureMapRFF(input_dim, dim=num_rff , gamma=gamma, random_state=random_state)
    psi_x = fm_x1(inputs)
    ones = tf.ones_like(inputs[:, 0:1])
    rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
    rho_x = tf.expand_dims(rho_x, axis=-1)
    qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=num_rff, dim_y=output_dim, n_comp=n_comp)
    rho_y = qmdmc(rho_x)
    y_w = rho_y[:, 0, :] # shape (b, d_in)
    y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
    probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
    qmdmclf = tf.keras.Model(inputs=inputs, outputs=probs)
    return qmdmclf

def create_model_rho(input_dim,
               output_dim,
               num_rff = 512,
               gamma = 2**-5,
               n_comp = 80,
               random_state = 0):
    inputs = tf.keras.Input(shape=(input_dim,))
    fm_x1 = layers.QFeatureMapRFF(input_dim, dim=num_rff , gamma=gamma, random_state=random_state)
    psi_x = fm_x1(inputs)
    ones = tf.ones_like(inputs[:, 0:1])
    rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
    rho_x = tf.expand_dims(rho_x, axis=-1)
    qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=num_rff, dim_y=output_dim, n_comp=n_comp)
    rho_y = qmdmc(rho_x)
    qmdmclf = tf.keras.Model(inputs=inputs, outputs=rho_y)
    return qmdmclf


In [21]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf_1 = create_model(784 // 2, 10, num_rff=256, n_comp=20)
qmdmclf_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 10
BATCH_SIZE = 256  
history = qmdmclf_1.fit(X_train[:, 0:784 // 2], y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 2s 7ms/step - loss: 1.5370 - accuracy: 0.5567 - val_loss: 0.6713 - val_accuracy: 0.8056
Epoch 2/10
188/188 [==============================] - 1s 7ms/step - loss: 0.6127 - accuracy: 0.8122 - val_loss: 0.5294 - val_accuracy: 0.8264
Epoch 3/10
188/188 [==============================] - 1s 7ms/step - loss: 0.4923 - accuracy: 0.8318 - val_loss: 0.4862 - val_accuracy: 0.8327
Epoch 4/10
188/188 [==============================] - 1s 8ms/step - loss: 0.4474 - accuracy: 0.8384 - val_loss: 0.4617 - val_accuracy: 0.8410
Epoch 5/10
188/188 [==============================] - 1s 7ms/step - loss: 0.4214 - accuracy: 0.8502 - val_loss: 0.4387 - val_accuracy: 0.8977
Epoch 6/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3800 - accuracy: 0.9072 - val_loss: 0.3797 - val_accuracy: 0.9112
Epoch 7/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3342 - accuracy: 0.9179 - val_loss: 0.3577 - val_accuracy: 0.9163
Epoch 

In [22]:
qmdmclf_2 = create_model(784 // 2, 10, num_rff=256, n_comp=20)
qmdmclf_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

history = qmdmclf_2.fit(X_train[:, 784 // 2:], y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 2s 8ms/step - loss: 1.0148 - accuracy: 0.7109 - val_loss: 0.4588 - val_accuracy: 0.8604
Epoch 2/10
188/188 [==============================] - 1s 7ms/step - loss: 0.4336 - accuracy: 0.8659 - val_loss: 0.4206 - val_accuracy: 0.8706
Epoch 3/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3891 - accuracy: 0.8799 - val_loss: 0.3983 - val_accuracy: 0.8768
Epoch 4/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3652 - accuracy: 0.8858 - val_loss: 0.3892 - val_accuracy: 0.8789
Epoch 5/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3505 - accuracy: 0.8901 - val_loss: 0.3747 - val_accuracy: 0.8845
Epoch 6/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3272 - accuracy: 0.8982 - val_loss: 0.3748 - val_accuracy: 0.8861
Epoch 7/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3261 - accuracy: 0.8992 - val_loss: 0.3653 - val_accuracy: 0.8869
Epoch 

## Multimodal model

In [117]:
inputs = tf.keras.Input(shape=(784,))
clf_1 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=10)
clf_2 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=10)
rho_1 = clf_1(inputs[:, :784 // 2])
rho_2 = clf_2(inputs[:, 784 // 2:])
prod = layers.DMCrossProduct()([rho_1, rho_2])
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=100, dim_y=10, n_comp=40)
rho_y = qmdmc(prod)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
qm_multim = tf.keras.Model(inputs=inputs, outputs=probs)



In [118]:
qm_multim.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 3
history = qm_multim.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/3
188/188 [==============================] - 9s 42ms/step - loss: 0.9769 - accuracy: 0.7044 - val_loss: 0.3627 - val_accuracy: 0.8944
Epoch 2/3
188/188 [==============================] - 8s 41ms/step - loss: 0.2897 - accuracy: 0.9314 - val_loss: 0.2296 - val_accuracy: 0.9445
Epoch 3/3
188/188 [==============================] - 8s 41ms/step - loss: 0.1859 - accuracy: 0.9599 - val_loss: 0.2147 - val_accuracy: 0.9500


In [29]:
qm_multim.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 784)]        0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_47 (Sl (None, 392)          0           input_23[0][0]                   
__________________________________________________________________________________________________
tf.__operators__.getitem_48 (Sl (None, 392)          0           input_23[0][0]                   
__________________________________________________________________________________________________
model_21 (Functional)           (None, 11, 10)       103278      tf.__operators__.getitem_47[0][0]
___________________________________________________________________________________________

## Noisy dataset

In [46]:
X_test_noise = np.array(X_test)
X_test_noise[:, :784 // 2] =  np.random.uniform(size=X_test.shape)[:, :784 // 2]
model.evaluate(X_test_noise, y_test_bin)

313/313 [==============================] - 0s 480us/step - loss: 9.5569 - categorical_accuracy: 0.2349


[9.556869506835938, 0.23489999771118164]

In [51]:
qm_multim.evaluate(X_test_noise, y_test_bin)

313/313 [==============================] - 1s 3ms/step - loss: 1.4351 - accuracy: 0.4704


[1.4350522756576538, 0.47040000557899475]

## Partial model

In [120]:
inputs = tf.keras.Input(shape=(784,))
clf_1 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=10)
clf_2 = layers.QMClassifSDecompFDMatrix()
rho_1 = clf_1(inputs[:, :784 // 2])
rho_2 = tf.keras.layers.concatenate((tf.ones((1, 10)) / 10., tf.eye(10)), axis = 0)
#rho_2 = tf.keras.layers.concatenate((rho_2, tf.zeros((11, 10))), axis=1)
rho_2 = tf.broadcast_to(rho_2, shape=tf.shape(rho_1))
prod = layers.DMCrossProduct()([rho_1, rho_2])
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=100, dim_y=10, n_comp=40)
rho_y = qmdmc(prod)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
qm_partial = tf.keras.Model(inputs=inputs, outputs=probs)
qm_partial.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

In [121]:
qm_partial.layers[2].set_weights(qm_multim.layers[3].get_weights())
qm_partial.layers[6].set_weights(qm_multim.layers[6].get_weights())

In [122]:
qm_partial.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 3ms/step - loss: 1.6581 - accuracy: 0.3551


[1.6514054536819458, 0.35740000009536743]

In [6]:
BATCH_SIZE = 256
input_dim = 784
component_dim = 128
gamma = 2**-5
n_comp = 80
random_state = 0

inputs = tf.keras.Input(shape=(784,))
fm_x = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
#qmdmc = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig, num_eig=num_eig)
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=component_dim, dim_y=10, n_comp=n_comp)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 10
  
history = qmdmclf.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 8s 42ms/step - loss: 1.6390 - accuracy: 0.4964 - val_loss: 0.5697 - val_accuracy: 0.8938
Epoch 2/10
188/188 [==============================] - 8s 42ms/step - loss: 0.5328 - accuracy: 0.8988 - val_loss: 0.4274 - val_accuracy: 0.9255
Epoch 3/10
188/188 [==============================] - 8s 42ms/step - loss: 0.3972 - accuracy: 0.9301 - val_loss: 0.3680 - val_accuracy: 0.9365
Epoch 4/10
188/188 [==============================] - 8s 41ms/step - loss: 0.3344 - accuracy: 0.9439 - val_loss: 0.3350 - val_accuracy: 0.9425
Epoch 5/10
188/188 [==============================] - 8s 42ms/step - loss: 0.2952 - accuracy: 0.9519 - val_loss: 0.3137 - val_accuracy: 0.9472
Epoch 6/10
188/188 [==============================] - 8s 42ms/step - loss: 0.2704 - accuracy: 0.9568 - val_loss: 0.2934 - val_accuracy: 0.9497
Epoch 7/10
188/188 [==============================] - 8s 42ms/step - loss: 0.2484 - accuracy: 0.9624 - val_loss: 0.2777 - val_accuracy: 0.9523

In [7]:
inputs = tf.keras.Input(shape=(784,))
fm_x1 = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x1(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc1 = layers.QMClassifSDecompFDMatrix(dim_x=component_dim, dim_y=10, n_comp=n_comp)
rho_h = qmdmc1(rho_x)
qmdmc2 = layers.QMClassifSDecompFDMatrix(dim_x=10, dim_y=10, n_comp=n_comp)
rho_y = qmdmc2(rho_h)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf2 = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

# We freeze the weights of the first layer and train the second
fm_x1.set_weights(fm_x.get_weights())
fm_x1.trainable = False
qmdmc1.set_weights(qmdmc.get_weights())
qmdmc1.trainable = False
EPOCHS = 5
  
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/5
188/188 [==============================] - 11s 53ms/step - loss: 1.0492 - accuracy: 0.9238 - val_loss: 0.2500 - val_accuracy: 0.9600
Epoch 2/5
188/188 [==============================] - 10s 55ms/step - loss: 0.1993 - accuracy: 0.9727 - val_loss: 0.2382 - val_accuracy: 0.9615
Epoch 3/5
188/188 [==============================] - 10s 55ms/step - loss: 0.1867 - accuracy: 0.9752 - val_loss: 0.2278 - val_accuracy: 0.9626
Epoch 4/5
188/188 [==============================] - 11s 56ms/step - loss: 0.1770 - accuracy: 0.9775 - val_loss: 0.2237 - val_accuracy: 0.9640
Epoch 5/5
188/188 [==============================] - 10s 52ms/step - loss: 0.1703 - accuracy: 0.9783 - val_loss: 0.2213 - val_accuracy: 0.9646


In [8]:
# We free all the weights and fine tune

fm_x1.trainable = True
qmdmc1.trainable = True
EPOCHS = 10
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 10s 53ms/step - loss: 0.1661 - accuracy: 0.9786 - val_loss: 0.2182 - val_accuracy: 0.9656
Epoch 2/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1606 - accuracy: 0.9796 - val_loss: 0.2116 - val_accuracy: 0.9663
Epoch 3/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1562 - accuracy: 0.9804 - val_loss: 0.2100 - val_accuracy: 0.9660
Epoch 4/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1512 - accuracy: 0.9812 - val_loss: 0.2074 - val_accuracy: 0.9654
Epoch 5/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1472 - accuracy: 0.9820 - val_loss: 0.2066 - val_accuracy: 0.9663
Epoch 6/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1433 - accuracy: 0.9826 - val_loss: 0.2030 - val_accuracy: 0.9676
Epoch 7/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1399 - accuracy: 0.9835 - val_loss: 0.2011 - val_accuracy: